In [24]:
import numpy as np
import invisible_cities.database.load_db as dbf

In [25]:
run_number = 4495
nsipms = 1792

In [26]:
sensor_ids = np.array([1023, 1601, 1609, 1687, 1403, 1376, 1379, 1384, 1385, 1386, 1387,
                       1392, 1393, 1394, 1395, 1396, 1397, 1400, 1401, 1402, 1431, 1454,
                       1455, 1462, 1471, 1125], dtype='i4')
charges    = np.array([ 2.66531368,  3.73287782,  5.2828832 ,  2.31351022,  4.41027829,
                        2.20720324,  3.98584685, 14.72511416, 18.30488489,  6.26837369,
                        2.96073255, 14.59842807, 23.50427029,  6.8357565 ,  3.92635856,
                        7.65335644,  2.09604995,  3.67059242,  9.99260022, 10.94429901,
                        2.83670564,  2.89119827,  2.57772977,  2.82898736,  2.83071036,
                        2.26349407], dtype='f4')

In [27]:
DataSiPM = dbf.DataSiPM(run_number)
DataPMT  = dbf.DataPMT(run_number)

## Create voxels

In [28]:
dist = 20.
sipm_dist = np.float32(20.)
sipm_thr = 5. # establish 
sizeX = 2.
sizeY = 2.
rMax = 198

In [29]:
#%%time
#Compute min,max x,y
#size x size y, rmax
selC = (charges > sipm_thr)
xmin = np.float32(DataSiPM.X[sensor_ids[selC]].values.min()-dist)
xmax = np.float32(DataSiPM.X[sensor_ids[selC]].values.max()+dist)
ymin = np.float32(DataSiPM.Y[sensor_ids[selC]].values.min()-dist)
ymax = np.float32(DataSiPM.Y[sensor_ids[selC]].values.max()+dist)
charge = np.float32(charges.mean())
xsize = np.float32(sizeX)
ysize = np.float32(sizeY)
rmax = np.float32(rMax)

#### Call CUDA kernel

In [7]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy
from pycuda.compiler import SourceModule

#from pycuda.tools import make_default_context
#cuda.init()
#ctx = make_default_context()

#compile cuda code
kernel_code = open('reset_non_compact.cu').read()
mod = SourceModule(kernel_code)
create_voxels = mod.get_function("create_voxels")

#TODO: Check rounding here
threads_x = int((xmax - xmin) / xsize)
threads_y = int((ymax - ymin) / ysize)
print(threads_x, threads_y)

#allocate memory for result
voxels_dt = np.dtype([('x', 'f4'), ('y', 'f4'), ('E', 'f4'), ('active', 'i4')])
num_voxels = threads_x * threads_y
voxels_d = cuda.mem_alloc(num_voxels * voxels_dt.itemsize)


%time create_voxels(voxels_d, xmin, xmax, ymin, ymax, xsize, ysize, rmax, charge, block=(1, 1, 1), grid=(threads_x, threads_y))

#ctx.detach()

40 40
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 953 µs


In [8]:
voxels_h = cuda.from_device(voxels_d, (num_voxels,), voxels_dt)

In [9]:
for h in voxels_h:
    print (h)

(115., 25., 6.396444, 1)
(115., 27., 6.396444, 1)
(115., 29., 6.396444, 1)
(115., 31., 6.396444, 1)
(115., 33., 6.396444, 1)
(115., 35., 6.396444, 1)
(115., 37., 6.396444, 1)
(115., 39., 6.396444, 1)
(115., 41., 6.396444, 1)
(115., 43., 6.396444, 1)
(115., 45., 6.396444, 1)
(115., 47., 6.396444, 1)
(115., 49., 6.396444, 1)
(115., 51., 6.396444, 1)
(115., 53., 6.396444, 1)
(115., 55., 6.396444, 1)
(115., 57., 6.396444, 1)
(115., 59., 6.396444, 1)
(115., 61., 6.396444, 1)
(115., 63., 6.396444, 1)
(115., 65., 6.396444, 1)
(115., 67., 6.396444, 1)
(115., 69., 6.396444, 1)
(115., 71., 6.396444, 1)
(115., 73., 6.396444, 1)
(115., 75., 6.396444, 1)
(115., 77., 6.396444, 1)
(115., 79., 6.396444, 1)
(115., 81., 6.396444, 1)
(115., 83., 6.396444, 1)
(115., 85., 6.396444, 1)
(115., 87., 6.396444, 1)
(115., 89., 6.396444, 1)
(115., 91., 6.396444, 1)
(115., 93., 6.396444, 1)
(115., 95., 6.396444, 1)
(115., 97., 6.396444, 1)
(115., 99., 6.396444, 1)
(115., 101., 6.396444, 1)
(115., 103., 6.396444, 1

(149., 63., 6.396444, 1)
(149., 65., 6.396444, 1)
(149., 67., 6.396444, 1)
(149., 69., 6.396444, 1)
(149., 71., 6.396444, 1)
(149., 73., 6.396444, 1)
(149., 75., 6.396444, 1)
(149., 77., 6.396444, 1)
(149., 79., 6.396444, 1)
(149., 81., 6.396444, 1)
(149., 83., 6.396444, 1)
(149., 85., 6.396444, 1)
(149., 87., 6.396444, 1)
(149., 89., 6.396444, 1)
(149., 91., 6.396444, 1)
(149., 93., 6.396444, 1)
(149., 95., 6.396444, 1)
(149., 97., 6.396444, 1)
(149., 99., 6.396444, 1)
(149., 101., 6.396444, 1)
(149., 103., 6.396444, 1)
(151., 25., 6.396444, 1)
(151., 27., 6.396444, 1)
(151., 29., 6.396444, 1)
(151., 31., 6.396444, 1)
(151., 33., 6.396444, 1)
(151., 35., 6.396444, 1)
(151., 37., 6.396444, 1)
(151., 39., 6.396444, 1)
(151., 41., 6.396444, 1)
(151., 43., 6.396444, 1)
(151., 45., 6.396444, 1)
(151., 47., 6.396444, 1)
(151., 49., 6.396444, 1)
(151., 51., 6.396444, 1)
(151., 53., 6.396444, 1)
(151., 55., 6.396444, 1)
(151., 57., 6.396444, 1)
(151., 59., 6.396444, 1)
(151., 61., 6.396444, 1

(169., 25., 6.396444, 1)
(169., 27., 6.396444, 1)
(169., 29., 6.396444, 1)
(169., 31., 6.396444, 1)
(169., 33., 6.396444, 1)
(169., 35., 6.396444, 1)
(169., 37., 6.396444, 1)
(169., 39., 6.396444, 1)
(169., 41., 6.396444, 1)
(169., 43., 6.396444, 1)
(169., 45., 6.396444, 1)
(169., 47., 6.396444, 1)
(169., 49., 6.396444, 1)
(169., 51., 6.396444, 1)
(169., 53., 6.396444, 1)
(169., 55., 6.396444, 1)
(169., 57., 6.396444, 1)
(169., 59., 6.396444, 1)
(169., 61., 6.396444, 1)
(169., 63., 6.396444, 1)
(169., 65., 6.396444, 1)
(169., 67., 6.396444, 1)
(169., 69., 6.396444, 1)
(169., 71., 6.396444, 1)
(169., 73., 6.396444, 1)
(169., 75., 6.396444, 1)
(169., 77., 6.396444, 1)
(169., 79., 6.396444, 1)
(169., 81., 6.396444, 1)
(169., 83., 6.396444, 1)
(169., 85., 6.396444, 1)
(169., 87., 6.396444, 1)
(169., 89., 6.396444, 1)
(169., 91., 6.396444, 1)
(169., 93., 6.396444, 1)
(169., 95., 6.396444, 1)
(169., 97., 6.396444, 1)
(169., 99., 6.396444, 1)
(169., 101., 6.396444, 1)
(169., 103., 6.396444, 1

(187., 35., 6.396444, 1)
(187., 37., 6.396444, 1)
(187., 39., 6.396444, 1)
(187., 41., 6.396444, 1)
(187., 43., 6.396444, 1)
(187., 45., 6.396444, 1)
(187., 47., 6.396444, 1)
(187., 49., 6.396444, 1)
(187., 51., 6.396444, 1)
(187., 53., 6.396444, 1)
(187., 55., 6.396444, 1)
(187., 57., 6.396444, 1)
(187., 59., 6.396444, 1)
(187., 61., 6.396444, 1)
(187., 63., 6.396444, 1)
(187., 65., 6.396444, 1)
(187., 67., 6.396444, 0)
(187., 69., 6.396444, 0)
(187., 71., 6.396444, 0)
(187., 73., 6.396444, 0)
(187., 75., 6.396444, 0)
(187., 77., 6.396444, 0)
(187., 79., 6.396444, 0)
(187., 81., 6.396444, 0)
(187., 83., 6.396444, 0)
(187., 85., 6.396444, 0)
(187., 87., 6.396444, 0)
(187., 89., 6.396444, 0)
(187., 91., 6.396444, 0)
(187., 93., 6.396444, 0)
(187., 95., 6.396444, 0)
(187., 97., 6.396444, 0)
(187., 99., 6.396444, 0)
(187., 101., 6.396444, 0)
(187., 103., 6.396444, 0)
(189., 25., 6.396444, 1)
(189., 27., 6.396444, 1)
(189., 29., 6.396444, 1)
(189., 31., 6.396444, 1)
(189., 33., 6.396444, 1

## Create anode response

#### Step 1: Initialize sipms with zero charge

In [10]:
kernel_code = open('reset_non_compact.cu').read()
mod = SourceModule(kernel_code)
initiliaze_anode = mod.get_function("initialize_anode")

In [11]:
xs = DataSiPM.X.values.astype('f4')
ys = DataSiPM.Y.values.astype('f4')

x_d = cuda.to_device(xs)
y_d = cuda.to_device(ys)

In [12]:
#allocate memory for result
sensors_dt = np.dtype([('id', 'i4'), ('charge', 'f4'), ('active', 'i1')])
sensors_d = cuda.mem_alloc(nsipms * sensors_dt.itemsize)

In [13]:
#%time initiliaze_anode(sensors_d, xmin, xmax, x_d, ymin, ymax, y_d, sipm_dist, block=(1, 1, 1), grid=(nsipms, 1))
initiliaze_anode(sensors_d, xmin, xmax, x_d, ymin, ymax, y_d, sipm_dist, block=(1, 1, 1), grid=(nsipms, 1))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 270 µs


#### Step 2: Put the correct charge for active sensors

In [14]:
kernel_code = open('reset_non_compact.cu').read()
mod = SourceModule(kernel_code)
create_anode_response = mod.get_function("create_anode_response")

In [15]:
sensor_ids_d = cuda.to_device(sensor_ids)
charges_d    = cuda.to_device(charges)

In [16]:
nsensors = sensor_ids.shape[0]
#%time create_anode_response(sensors_d, sensor_ids_d, charges_d, block=(1, 1, 1), grid=(nsensors, 1))
create_anode_response(sensors_d, sensor_ids_d, charges_d, block=(1, 1, 1), grid=(nsensors, 1))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 170 µs


In [17]:
sensor_ids

array([1023, 1601, 1609, 1687, 1403, 1376, 1379, 1384, 1385, 1386, 1387,
       1392, 1393, 1394, 1395, 1396, 1397, 1400, 1401, 1402, 1431, 1454,
       1455, 1462, 1471, 1125], dtype=int32)

In [18]:
charges

array([ 2.6653137,  3.7328777,  5.282883 ,  2.3135102,  4.4102783,
        2.2072031,  3.9858468, 14.725114 , 18.304884 ,  6.2683735,
        2.9607325, 14.598428 , 23.50427  ,  6.8357563,  3.9263585,
        7.6533566,  2.09605  ,  3.6705923,  9.9926   , 10.944299 ,
        2.8367057,  2.8911982,  2.5777297,  2.8289874,  2.8307104,
        2.263494 ], dtype=float32)

## Select active sensors

In [22]:
active_dt = np.dtype([('id', 'i1')])
active_d = cuda.mem_alloc(num_voxels * nsipms) # TODO: Update after compact
#probabilities = cuda.mem_alloc(nvoxels * nsensors)

In [23]:
kernel_code = open('reset_non_compact.cu').read()
mod = SourceModule(kernel_code)
compute_active_sensors = mod.get_function("compute_active_sensors")

LogicError: cuModuleLoadDataEx failed: an illegal memory access was encountered - 

In [21]:
%time compute_active_sensors(active_d, sensors_d, voxels_d, np.int32(nsipms), sipm_dist, block=(1, 1, 1), grid=(num_voxels, 1))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 234 µs


In [31]:
DataSiPM[1125:1126]

,SensorID,ChannelID,Active,X,Y,adc_to_pes
1125,18037,18040,1,45.0,-25.0,16.857


In [34]:
print(xmin, xmax)
print(ymin, ymax)

115.0 195.0
25.0 105.0
